<a href="https://colab.research.google.com/github/123shwetarohokale/563-ShwetaR/blob/main/Robotics_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def skew_symmetric(omega):
    """Returns the 3x3 skew-symmetric matrix of a vector."""
    wx, wy, wz = omega
    return np.array([
        [ 0, -wz,  wy],
        [ wz,  0, -wx],
        [-wy, wx,   0]
    ])

def omega_matrix(omega):
    """Constructs the 4x4 Omega(omega) matrix for quaternion kinematics."""
    omega_cross = skew_symmetric(omega)
    top = np.hstack(([0], -omega))
    bottom = np.hstack((omega[:, np.newaxis], -omega_cross))
    return np.vstack((top[np.newaxis, :], bottom))

def closed_form_quaternion_update(q, omega, dt):
    """
    Updates quaternion using closed-form exponential of Omega matrix.

    Parameters:
    q     -- current quaternion [q0, q1, q2, q3] (scalar-first)
    omega -- angular velocity vector [wx, wy, wz] in rad/s
    dt    -- time interval in seconds

    Returns:
    q_next -- updated quaternion (normalized)
    """
    norm_omega = np.linalg.norm(omega)
    if norm_omega == 0:
        return q

    theta = norm_omega * dt
    Omega = 0.5 * omega_matrix(omega)
    I = np.eye(4)

    exp_Omega_dt = (
        np.cos(theta / 2) * I +
        (np.sin(theta / 2) / norm_omega) * Omega
    )

    q_next = exp_Omega_dt @ q
    return q_next / np.linalg.norm(q_next)

def run_simulation():
    # Initial quaternion (no rotation)
    q = np.array([1.0, 0.0, 0.0, 0.0])
    q_initial = q.copy()

    # Time step
    dt = 0.25  # seconds

    # Define the sequence of angular velocities
    rotations = [
        ("Rotate +π rad/s about X",  np.array([ np.pi, 0.0, 0.0])),
        ("Rotate +π rad/s about Z",  np.array([0.0, 0.0,  np.pi])),
        ("Rotate −π rad/s about X",  np.array([-np.pi, 0.0, 0.0])),
        ("Rotate −π rad/s about Z",  np.array([0.0, 0.0, -np.pi]))
    ]

    print("Initial quaternion q(0):", q)

    # Apply each rotation
    for step_name, omega in rotations:
        q = closed_form_quaternion_update(q, omega, dt)
        print(f"After '{step_name}': q = {np.round(q, decimals=6)}")

    # Final result
    print("\nFinal quaternion q(1):", np.round(q, 6))
    print("Initial quaternion q(0):", q_initial)

    # Check if final quaternion is approximately equal to the initial
    if np.allclose(q, q_initial, atol=1e-6) or np.allclose(q, -q_initial, atol=1e-6):
        print("Result: ✅ Final orientation is the same as the initial orientation (modulo sign).")
    else:
        print("Result: ❌ Final orientation is different from the initial orientation.")

if __name__ == "__main__":
    run_simulation()


Initial quaternion q(0): [1. 0. 0. 0.]
After 'Rotate +π rad/s about X': q = [0.97922  0.202803 0.       0.      ]
After 'Rotate +π rad/s about Z': q = [ 0.958871  0.198589 -0.041129  0.198589]
After 'Rotate −π rad/s about X': q = [ 0.97922   0.       -0.080549  0.186121]
After 'Rotate −π rad/s about Z': q = [ 0.996617  0.016336 -0.078875 -0.016336]

Final quaternion q(1): [ 0.996617  0.016336 -0.078875 -0.016336]
Initial quaternion q(0): [1. 0. 0. 0.]
Result: ❌ Final orientation is different from the initial orientation.
